<a href="https://colab.research.google.com/github/Roymido97/Geological-Lithology-classification-Kansas-well-logs/blob/main/Geological_Facies_ML_Classification_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lithology facies Machine Learning Classification: Using SMOTE, GridSearch, and Ensemble.


# **Introduction and Setup**

This notebook explores a dataset related to facies classification in well logs. Facies, representing different rock types, are crucial for understanding subsurface geology and reservoir characterization. The goal is to build a machine learning model capable of accurately predicting facies from well log data.

The labeled well log data are from real wells of Council Grove gas reservoir in Southwest Kansas. The Panoma Council Grove Field is predominantly a carbonate gas reservoir encompassing 2700 square miles in Southwestern Kansas.

There are totally seven features for each well log:

gamma ray (GR)

resistivity logging (ILD_log10)

photoelectric effect (PE)

neutron-density porosity difference
average neutron-density porosity (DeltaPHI and PHIND)

nonmarine-marine indicator (NM_M)

and relative position (RELPOS)

![image.png](attachment:43f7895a-d879-424e-811a-fbf469c75f57.png)


We begin by importing the necessary libraries for data manipulation, visualization, and machine learning.


# Importing the necessary libraries

In [ ]:
# for DataFrame operation
import numpy as np
import pandas as pd

# for visualizations
import seaborn as sns
import matplotlib.pyplot as plt

# for preprocessing
from sklearn.impute import KNNImputer
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

# for ML modeling
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier

#for Neural Network
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# for model evaluation
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay

## 1. Data Loading and Initial Inspection

Next, we load the dataset from a Google Drive URL. for more generalized approach.

In [ ]:
# Load the dataset (Google Drive)
url = 'https://drive.google.com/file/d/19UU1UL6J-VfnV29AQoVlpWaqoVOdywPO/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id=' + url.split('/')[-2]


data = pd.read_csv(path)


# --- Initial Data Inspection ---
print("--- Initial Data Inspection ---")
print(data.info())
print("\n--- Missing Values ---")
print(data.isnull().sum())
print("\n--- First 5 Rows ---")
print(data.head())

## 2. Exploratory Data Analysis (EDA)


## 2.1. Descriptive Statistics

In [ ]:
print(data.describe())

## 2.2. Class Distribution (Target Variable)

This is crucial to identify class imbalances. If some classes are significantly under-represented, it may be necessary to use techniques like SMOTE.


In [ ]:
print("\n--- Class Distribution (Target Variable) ---")
print(data['Facies'].value_counts())
sns.countplot(x='Facies', data=data)
plt.title('Distribution of Facies')
plt.show()

### observation
- The Facies have a great imbalance, can be remedied using SMOTE

## 2.3. Correlation Analysis

Look for highly correlated features, which might indicate multicollinearity.  This can impact the performance of some models.

In [ ]:
# Identifying categorical valuables
data.head()

In [ ]:
# Drop 'Formation' and 'Well Name' columns
data = data.drop(columns=['Formation', 'Well Name'])

In [ ]:
print("\n--- Correlation Analysis ---")
plt.figure(figsize=(12, 8))  # Adjust figure size for better readability
sns.heatmap(data.corr(), annot=True, cmap='coolwarm', fmt=".1f")
plt.title('Correlation Heatmap')
plt.show()

## 2.4. Feature Distributions
This helps you understand the distribution of each feature, identify skewness, and potential outliers.


In [ ]:

# Histograms for numerical features
print("\n--- Feature Distributions ---")

Numerical=data[['Depth','GR','ILD_log10','DeltaPHI','PE','PHIND']]
Numerical.hist(figsize=(10, 12), bins=30)
plt.tight_layout()
plt.show()

### Observations:
- the mean of Gamma Ray logs lay between (60 and 80).
- the mean of photoelectric effect logs lay between (3 and 4).
- the mean of resistivity logs lay between (0.50 and 0.75).
- The depth in study is between 2600 to 3100 ft.

In [ ]:
# comparing Facies across diff Logs

logs = {
    'GR': ('Mean Gamma Ray', 'Mean Gamma Ray per Facies'),
    'ILD_log10': ('Mean Resistivity', 'Mean Resistivity Log per Facies'),
    'PE': ('Mean Photoelectric Effect', 'Mean Photoelectric Effect Log per Facies'),
    'PHIND': ('Mean Neutron-Density Porosity', 'Mean Neutron-Density Porosity Log per Facies')
}

# Create a 2x2 grid of subplots
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))


axes = axes.flatten()


for i, (log_column, (ylabel, title)) in enumerate(logs.items()):
    log_per_facies = data.groupby('Facies')[log_column].mean()
    log_per_facies.plot(kind='bar', ax=axes[i], xlabel='Facies', ylabel=ylabel, title=title, color='c' )
    axes[i].tick_params(axis='x', rotation=0)

# Adjust layout and display

plt.tight_layout()
plt.show()

### Observations:
- Gamma Ray:
     - The highest content: face 4 (Marine Siltstone and Shale), which is predictable.
     - The lowest content: face 9 (Bafflestone).
       
- Restivity:

      - The highest content: face 6 (Mudstone).
      - The lowest content: face 1 (Nonmarine Sandstone)

- Photoelectric:

      - The highest content: face 9 (Bafflestone).
      - The lowest content: face 1 (Nonmarine Sandstone).

- Neutron-Density Porosity:

      - The highest content: face 3 (Nonmarine fine Siltstone).
      - The lowest content: face 6 (Mudstone).

## 3. Data Preprocessing


In [ ]:
# Handle missing values using KNN imputation
print("\n--- Handling missing values with KNN Imputation ---")
imputer = KNNImputer(n_neighbors=5)
data_imputed = imputer.fit_transform(data)  # Apply to the entire dataset

In [ ]:
# Convert back to a DataFrame
data_imputed = pd.DataFrame(data_imputed, columns=data.columns)
print(data_imputed.info())
print(data_imputed.isnull().sum())  # Verify that missing values are handled.

In [ ]:
# Define features and target
features = ['Depth','GR', 'ILD_log10','DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']
X = data_imputed[features]
y = data_imputed['Facies']

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Handle imbalanced data using SMOTE

print("\n--- Handling imbalanced data using SMOTE ---")
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
# Standardize features

print("\n--- Standardizing features ---")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

## 4. Model Training and Evaluation


In [ ]:
#  models

models = {
    'SVM': SVC(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'NB': GaussianNB(),
    'DT': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(),
    'Log Reg': LogisticRegression(max_iter=1000),
    'RF': RandomForestClassifier()
}

In [ ]:
# Hyperparameter tuning using GridSearchCV

param_grids = {
    'SVM': {
        'C': [ 5,10],  # Regularization parameter
        'gamma': [0.01, 0.1 ],  # Kernel coefficient
        'kernel': ['rbf', 'sigmoid']  # Kernel type
    },
    'Gradient Boosting': {
        'n_estimators': [ 100, 200],  # Number of boosting stages
        'learning_rate': [0.05, 0.1],  # Learning rate
        'max_depth': [5, 10]  # Maximum depth of the trees
    },
    'NB': {
        'var_smoothing': [ 1e-7,1e-6]  # Smoothing parameter
    },
    'DT': {
        'max_depth': [ 10, 20],  # Maximum depth of the tree
        'min_samples_split': [2, 10],  # Minimum samples required to split a node
        'criterion': ['gini', 'entropy']  # Splitting criterion
    },
    'KNN': {
        'n_neighbors': [2, 4, 8],  # Number of neighbors (even values for odd categories, 9 facies)
        'weights': ['uniform', 'distance'],  # Weight function
        'algorithm': ['auto', 'ball_tree', 'kd_tree']  # Algorithm used to compute nearest neighbors
    },
    'Log Reg': {
        'C': [0.1,  10],  # Inverse of regularization strength
        'solver': ['lbfgs', 'liblinear']  # Optimization algorithm
    },
    'RF': {
        'n_estimators': [100, 500],  # Number of trees in the forest
        'max_depth': [ 10, 20],  # Maximum depth of the trees
        'criterion': ['gini', 'entropy']  # Splitting criterion
    }
}

In [ ]:
# GridSearchCV

for model_name, model in models.items():
    print(f"Tuning hyperparameters for {model_name}...")

    grid_search = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_scaled, y_resampled)

    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    print(f"Best cross-validation score for {model_name}: {grid_search.best_score_:.4f}\n")

In [ ]:

best_model_name = 'RF'
best_model = RandomForestClassifier(
    criterion='entropy',
    max_depth=20,
    n_estimators=500
)

best_model.fit(X_scaled, y_resampled)

y_pred = best_model.predict(X_test_scaled)

# confusion matrix

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=best_model.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title(f'Confusion Matrix for {best_model_name}')
plt.show()


In [ ]:
# Convert classes to strings

target_names = [str(cls) for cls in best_model.classes_]

# classification report

print(f"Classification Report for {best_model_name}:\n")
print(classification_report(y_test, y_pred, target_names=target_names))